<a href="https://colab.research.google.com/github/reykel/machine_learning_keras_tf_owned/blob/main/01-data_augmentation_dogs_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow_datasets as tfds
from tensorflow import keras
from keras import layers
from keras import callbacks

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
(train_examples, validation_examples, test_examples), info = tfds.load(
    'cats_vs_dogs',
    shuffle_files=True, 
    with_info=True, 
    as_supervised=True, 
    split=['train[:80%]', 'train[80%:95%]', 'train[95%:]'],
    )

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [ ]:
BATCH_SIZE = 100
IMG_SIZE = 150
EPOCHS = 100

In [ ]:
train_ds = train_examples.map(lambda x, y: (tf.image.resize(x, (IMG_SIZE, IMG_SIZE)), y))
val_ds = validation_examples.map(lambda x, y: (tf.image.resize(x, (IMG_SIZE, IMG_SIZE)), y))
test_ds = test_examples.map(lambda x, y: (tf.image.resize(x, (IMG_SIZE, IMG_SIZE)), y))

In [ ]:
data_augmentation = keras.Sequential(
    [layers.RandomFlip('horizontal'), 
    layers.RandomRotation(factor=(-0.025, 0.025)),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomContrast(factor=0.1),
])

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5, restore_best_weights=True)

In [ ]:
train_ds = train_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
                      
test_ds = test_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential([
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
model.build([None, IMG_SIZE, IMG_SIZE, 3])
model.summary()

In [ ]:
model.compile(
  optimizer= tf.keras.optimizers.Adam(0.001),
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, verbose=1, callbacks =[earlystopping])

In [ ]:
image_batch, label_batch = next(iter(test_ds.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

In [ ]:
predicted_batch = model.predict(image_batch)

In [ ]:
predicted_ids = np.argmax(predicted_batch, axis=-1)

for i in range(0,len(predicted_ids)):
  print(str(predicted_ids[i]) + " - " + str(label_batch[i]))

In [ ]:
cm = confusion_matrix(label_batch, predicted_ids)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['Cats','Dogs']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')